In [1]:
import sys
sys.path.append('C:\\Users\\DELL\\Graduation Project\\Codes and Scripts\\type-identification-mri-sequences\\')

In [2]:
#scikit-learn must be at version <= 0.17
import os
import argparse
import random
import cv2
import time
import matplotlib.pyplot as plt
import glob
import pandas  

import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision

import tensorflow as tf
from tensorflow.keras.utils import plot_model
import tensorflow_docs.vis.embed as embed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Conv2D,Conv2DTranspose,LeakyReLU,Activation,Concatenate,Add
import tensorflow.keras.layers as L
from tensorflow.keras import layers 

from models import select_net
from time_util import time_format
from MedicalDataset import MedicalDataset

from HelperFunctions import preprocess_image_train, generate_images_GIF, predict_image
from SynthModels import SqueezeAttention, ResNet, unet_model, Discriminator, old_squeeze

C:\Users\DELL\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


# Making the CSV File babe <3

In [3]:
# r"C:\Users\DELL\Desktop\BraTS2021_01250"
test_path = r"C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS2021_00452"
test_data_list = sorted(glob.glob(test_path + '/*'))

#delete segmentation(labels)
for test_data in test_data_list:
    if test_data.endswith('seg.nii.gz'):
        os.remove(test_data)        
test_data_list = sorted(glob.glob(test_path + '/*'))
        
num_rows = len(test_data_list)
test_data = pd.DataFrame({'path': test_data_list, 'label': np.nan})

In [4]:
test_data

,path,label
0,C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS20...,NaN
1,C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS20...,NaN
2,C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS20...,NaN


In [5]:
def fix_random_seeds():
    torch.backends.cudnn.deterministic = True
    random.seed(1)
    torch.manual_seed(1)
    torch.cuda.manual_seed(1)
    np.random.seed(1)
    
# args = parse_args()
# test_data_path = r"C:\Users\DELL\Graduation Project\Codes and Scripts\type-identification-mri-sequences\test.csv"
model_file = r"C:\Users\DELL\Graduation Project\Codes and Scripts\type-identification-mri-sequences\models\b4_sl4.pth"
n_slices = 4
tridim = False
consider_other_class = not '--no-other'
architecture = 'resnet18'

assert(architecture in ['resnet18', 'alexnet', 'vgg', 'squeezenet', 'mobilenet'])

fix_random_seeds()

test_set = MedicalDataset(test_data, min_slices = n_slices, consider_other_class = consider_other_class, test = True)
test_loader = data.DataLoader(test_set, num_workers = 8, pin_memory = True)

n_test_files = test_set.__len__()
classes = ['FLAIR', 'T1', 'T1c', 'T2']
paths = []
net = select_net(architecture, n_slices, tridim, consider_other_class)

if torch.cuda.is_available():
    net = net.cuda()

start_time = time.time()

#test
net.load_state_dict(torch.load(os.path.join('models', model_file), map_location=torch.device('cpu')))
net.eval()
correct = 0
total = 0
correct_per_class = [0] * len(classes)
total_per_class = [0] * len(classes)
actual_classes = []
predicted_classes = []
wrong_predictions = []
flag = 0
with torch.no_grad():
    for i, (pixel_data, label, path) in enumerate(test_loader):
        print('Tested', i + 1, 'of', n_test_files, 'files.')
        label_as_num = label.numpy()[0]
        if tridim:
            pixel_data = pixel_data.view(-1, 1, 10, 200, 200)

        outputs = net(pixel_data) #.cuda()
        _, predicted = torch.max(outputs.data, 1)

        predicted_classes.append(classes[predicted.cpu().numpy()[0]])
        paths.append(path[0])
        flag += 1
        if flag == 4:
#             predicted_classes.clear()
#             paths.clear()
            flag = 0


    if set(predicted_classes) == set(classes):
        print("ALL GOOD.\nOUT FOR SEGMENTATION\n")
    else:
        print("NOPE MISSING MODES.\nOUT FOR DATA CONVERSION\n")
        inter = list(set(predicted_classes).intersection(set(classes)))
        print("AVAILABLE MODES: {}".format(inter))

    print("\nPrediction:  ", predicted_classes)
    print("Paths: ", paths)

Starting loading data...
Loading C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS2021_00452\BraTS2021_00452_flair.nii.gz
Loaded 1 / 3 (counting discarded).
Loading C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS2021_00452\BraTS2021_00452_t1ce.nii.gz
Loaded 2 / 3 (counting discarded).
Loading C:\Users\DELL\Desktop\SqueezeAtt-30 T2 BraTS2021_00452\BraTS2021_00452_t2.nii.gz
Loaded 3 / 3 (counting discarded).

Loading time: 0h0min0s
Actually loaded: 3 ("Other" class discarded)
Tested 1 of 3 files.
Tested 2 of 3 files.
Tested 3 of 3 files.
NOPE MISSING MODES.
OUT FOR DATA CONVERSION

AVAILABLE MODES: ['T2', 'T1c', 'FLAIR']

Prediction:   ['FLAIR', 'T1c', 'T2']
Paths:  ['C:\\Users\\DELL\\Desktop\\SqueezeAtt-30 T2 BraTS2021_00452\\BraTS2021_00452_flair.nii.gz', 'C:\\Users\\DELL\\Desktop\\SqueezeAtt-30 T2 BraTS2021_00452\\BraTS2021_00452_t1ce.nii.gz', 'C:\\Users\\DELL\\Desktop\\SqueezeAtt-30 T2 BraTS2021_00452\\BraTS2021_00452_t2.nii.gz']


In [6]:
print(f"Classes: {classes}")
print(f"Predicted Classes: {predicted_classes}")
missing = set(classes) - set(predicted_classes)
print("Missing:", list(missing))

Classes: ['FLAIR', 'T1', 'T1c', 'T2']
Predicted Classes: ['FLAIR', 'T1c', 'T2']
Missing: ['T1']


In [7]:
if len(missing) > 1:
    raise Exception("wth is wrong bro?!")    

if len(missing) < 1:
    raise Exception("wtf is wrong bro?!")     

In [8]:
brats_num = int(os.path.basename(test_path)[-5:])
results_path = r'E:\Graduation Project\GIFs and Models\Brats {}\From Identification to Synthesize\imgs\imgs'.format(brats_num)
if not os.path.exists(results_path):
    os.makedirs(results_path)
    
modules_list = test_data_list
missing = missing.pop()

if missing == 'T1':
    generator_seq = 'T2'
    seq_1 = 'T1'
    seq_2 = 'T2'
    
if missing == 'T2':
    generator_seq = 'T1'
    seq_1 = 'T1'
    seq_2 = 'T2'
    
if missing == 'FLAIR':
    generator_seq = 'T2'
    seq_1 = 'T2'
    seq_2 = 'FLAIR'
    
vol_path = modules_list[predicted_classes.index(generator_seq)]    
vol_nifti = nib.load(vol_path).get_fdata()                    

for i in range(vol_nifti.shape[2]):
    nifti_slice = vol_nifti[:, :, i]
    rslt_path = f"{results_path}/{i:03d}.png"
    imageio.imwrite(rslt_path, nifti_slice)

Lossy conversion from float64 to uint8. Range [-1.0, 0.8847090005874634]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.921755313873291]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9713090062141418]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9801654815673828]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9849767684936523]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9638697504997253]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9714444279670715]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float6

Lossy conversion from float64 to uint8. Range [-1.0, 0.8384301662445068]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.7869566679000854]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.796642541885376]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.7836421728134155]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.8231570720672607]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.7710874080657959]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9175304174423218]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float6

Lossy conversion from float64 to uint8. Range [-1.0, 0.965795636177063]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9594064354896545]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9672890901565552]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9778462648391724]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9705227613449097]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9659144878387451]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [-1.0, 0.9691604375839233]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float6

In [10]:
if missing == 'T1':
    generator_g = SqueezeAttention().model
    generator_f = SqueezeAttention().model
    checkpoint_path = r"E:\Graduation Project\GIFs and Models\Squeeze Attention U-Net-30"

elif missing == 'T2':
    generator_g = SqueezeAttention().model
    generator_f = SqueezeAttention().model
    checkpoint_path = r"E:\Graduation Project\GIFs and Models\Squeeze Attention U-Net-30"

elif missing == 'FLAIR':
    generator_g = ResNet().model
    generator_f = ResNet().model
    checkpoint_path = r"E:\Graduation Project\GIFs and Models\ResNet 24-2-2023 T2-Flair-30"

discriminator_x = Discriminator().model
discriminator_y = Discriminator().model

generator_g_optimizer = tf.keras.optimizers.Adam(8e-5) 
generator_f_optimizer = tf.keras.optimizers.Adam(8e-5)
discriminator_x_optimizer = tf.keras.optimizers.Adam(8e-5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(8e-5)

In [13]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
data_path = os.path.dirname(results_path)

img_data = tf.keras.preprocessing.image_dataset_from_directory(
                          data_path,
                          seed=123,
                          image_size=(256, 256),
                          batch_size=1,
                          shuffle = False)
img_data = img_data.cache().prefetch(buffer_size=AUTOTUNE)
img_data = img_data.map(lambda x, _: (HelperFunctions.preprocess_image_train(x)))

Found 134 files belonging to 1 classes.


In [14]:
ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print(f'Last Check Point: {ckpt_manager.latest_checkpoint} is restored')

Last Check Point: E:\Graduation Project\GIFs and Models\Squeeze Attention U-Net-30\ckpt-30 is restored


In [18]:
vol = []
dep = len(glob.glob(results_path + '/*'))

In [19]:
for image in tf.data.Dataset.zip(img_data):
    img = predict_image(image, generator_f)
    vol.append(img)

In [17]:
for image in tf.data.Dataset.zip(img_data):
    if missing == 'T1':
        img = predict_image(image, generator_f)
        
    elif missing == 'T2':
        img = predict_image(image, generator_g)
        
    elif missing == 'FLAIR':
        img = predict_image(image, generator_f)
        
    vol.append(img)

In [21]:
original_vol_path = sorted(glob.glob(test_path + '/*'))[0]
original_vol = nib.load(original_vol_path)
original_shape = original_vol.shape

vol = np.array(vol).transpose(1, 2, 0)


vol = ndimage.zoom(vol, (original_shape[0]/vol.shape[0],
                                           original_shape[1]/vol.shape[1],
                                           original_shape[2]/vol.shape[2]), order=0)

v = nib.Nifti1Image(np.array(vol), original_vol.affine)            # to save this 3D (ndarry) numpy
    

if missing == 'T1':
    nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t1.nii.gz')

elif missing == 'T2':
    nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_t2.nii.gz')

elif missing == 'FLAIR':
    nib.save(v, test_path + '/' + f'BraTS2021_0{brats_num:04d}_flair.nii.gz')